In [281]:
# https://www.kaggle.com/del=59c8c55006eb2533/nips-papers-visualized-with-nmf-and-t-sne
import pandas as pd
import re
from datetime import datetime
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

from gensim.summarization.summarizer import summarize
import warnings
warnings.filterwarnings("ignore")

from PyKomoran import *

komoran = Komoran(DEFAULT_MODEL['FULL'])
komoran.set_user_dic('D:/petition/user_dic.dic')
# komoran.setUserdic('D:/petition/user_dic.dic')

def komoran_tokenizer(sent):
    words = komoran.get_list(sent)
    words = [str(w) for w in words if ('/NN' in str(w) or '/XR' in str(w) or '/VA' in str(w) or '/VV' in str(w))]
    return ' '.join([word.split("/")[0] for word in words])

def komoran_tokenizer(sent):
    words = komoran.get_list(sent)
    words = [str(w) for w in words if ('/NN' in str(w) in str(w))]
    return ' '.join([word.split("/")[0] for word in words])


data = pd.read_pickle("../../petition/petition_total_data.pkl")
data_add = pd.read_pickle("../../petition/total_data.pkl")
data = pd.concat([data,data_add], axis = 0)
data = data.reset_index(drop=True)
data.shape

(40667, 10)

In [282]:
data['sdays'] = pd.to_datetime(data['sdays'])
data['edays'] = pd.to_datetime(data['edays'])

data['count'] = pd.to_numeric(data['count'])

# 2019.03.31 이후 sns 계정에서 100명 이상의 동의를 받은 후에만 청원 글 게시 가능 

In [283]:
print(f" 가장 많은 동의 수 {data['count'].max()}, 가장 적은 동의 수 {data['count'].min()}")

 가장 많은 동의 수 2715626.0, 가장 적은 동의 수 0.0


In [284]:
data = data[data['sdays']>=datetime.strptime("2019.04.01", "%Y.%m.%d")]
data = data[data['sdays']<datetime.strptime("2021.04.01", "%Y.%m.%d")]

print(f" 가장 많은 동의 수 {data['count'].max()}, 가장 적은 동의 수 {data['count'].min()}")

 가장 많은 동의 수 2715626.0, 가장 적은 동의 수 107.0


In [285]:
data.shape

(17837, 10)

In [286]:
data.columns

Index(['code', 'sdays', 'edays', 'title', 'count', 'content', 'category',
       'progress', 'link', 'person'],
      dtype='object')

In [287]:
data['progress'].value_counts()

청원종료       17479
청원진행중        222
\n답변완료       119
답변완료          17
Name: progress, dtype: int64

In [288]:
data = data.drop_duplicates(['title'])

In [289]:
data.shape

(17777, 10)

In [290]:
print(f" 가장 많은 동의 수 {data['count'].max()}, 가장 적은 동의 수 {data['count'].min()}")

 가장 많은 동의 수 2715626.0, 가장 적은 동의 수 107.0


In [291]:
data = data[data['progress']=='청원진행중 ']

In [292]:
data = data.drop_duplicates("title")
data.shape

(221, 10)

In [293]:
emoji_pattern = re.compile("["
        u"\U0001F000-\U0001FFFF"  # 1F000~1FFFF 추가 
        u"\U000F0850-\U000F0853"               "]+", flags=re.UNICODE)


delete_pattern = ['\n','\t','\r', "\n\t\t\t\t\t\t\t\t\t\t",r"\r\n", '국민청원', '청와대','안녕하세요', "감사합니다"
                  'https?://[A-Za-z0-9./-]+',u'\xa0',u'\x0b',u'\\xa0', ' ■', '   ',"※","★","-"]

delete_pattern2 = "[본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]"

data['content'] = [emoji_pattern.sub(r'', content) for content in data['content']]
data['content'] = [re.sub('|'.join(delete_pattern),u' ',''.join(content)).strip().replace(delete_pattern2," ") for content in data['content']]
data = data[~data['content'].isin(['반대합니다','찬성합니다','동의합니다','동참합니다'])]

In [294]:
data = data.reset_index(drop=True)

In [295]:
data["content"] = data['content'].map(lambda x:x.replace("\\xa0",""))
data["prior_content"] = data['content'].map(lambda x : x.split(".")[0] + x.split(".")[len(x.split("."))-1])
data["for_content"] = data['content'].map(lambda x : ' '.join(x.split(".")[1:len(x.split("."))-2]))

In [296]:
data = data.reset_index(drop =True)

data['summary_content'] = ''

count = 1

for i in range(len(data)):
    
    # 본문이 특정 개수 문장 이하일 경우 결과가 반환되지 않음.
    # 이때는 요약하지 않고 본문에서 앞 3문장을 사용함.
        
    try:
        
        data['summary_content'][i] = summarize(data['for_content'][i], word_count= 0.15)
        
    except ValueError:
        
        data['summary_content'][i] = None
        
    if not data['summary_content'][i]:
        
        df_sentences =  data['for_content'][i].split('.')

        if len(df_sentences) > 3:

            data['']
            data['summary_content'][i] = '.'.join(df_sentences[:3])
            

        else:
            data['summary_content'][i] = '.'.join(df_sentences)
            
            count += 1

In [297]:
data["summary_content_end"] = data['prior_content'] + data['summary_content'] + data['title']

In [298]:
data["summary_content_end"] = data["summary_content_end"].map(lambda x: x.replace("\\",""))

In [299]:
import re
 
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
 
    return text
 
data["summary_content_end"] = data["summary_content_end"].map(lambda x: re.sub(r'[0-9]+', '',cleanText(x)))

In [300]:
for i in range(10):
    print("\n")
    print(data["summary_content_end"][i])
    print("--"*10)



코로나 거리두기 체계 개선해주세요소수 국민의 일탈방심을 막아 코로나확산을 막다 제안 요약자영업자•특정 업종 영업제한 정지모임 금지이동 제한를 시키는 것이 아니라거리두기단계에 따른 다중이용업소집합시설공공시설 등의 방역수칙마스크쓰기사회적거리두기을 위반한 방역위반자에게 과태료벌금를 단계에 따라만원단계 만원단계 만원단계부과한다 방역위반자의 방역수칙 과태료 규제단계 이후에집합시설단계부터집합시설 제한은단계로 확실하고 짧게 효과를 보도록 한다 방역위반자 마스크를 벗은 체불량 착용사회적거리두기m위반자마스크 정확한 착용 위반자 제안추천 거리두기 단계 예시최소 피해로 확실한 효과주 평균일세부 내용 생략단계 전국 확진자명 미만종식 전까지방역위반자 과태료만원단계 전국 확진자명 이상방역위반자 과태료만원단계 전국 확진자명 이상방역위반자 과태료만원단계 전국 확진자명 이상방역위반자 과태료만원현재 거리두기단계 방역조치단계 전국 확진자명 이상방역위반자 과태료만원현재 거리두기단계 방역조치 제안검증 단계 과도기 단계현재년월의 거리두기 단계별 방역위반자 과태료 부과세부내용 생략확진자 숫자는 대략적단계 전국 확진자명 미만종식 전까지방역위반자 과태료만원 단계 전국 확진자명 이상방역위반자 과태료만원단계 전국 확진자명 이상방역위반자 과태료만원 단계 전국 확진자명 이상방역위반자 과태료만원단계 전국 확진자명 이상방역위반자 과태료만원          코로나해결 방안   코로나전쟁의 승리거리두기 단계 조정 “소수 국민의 일탈방심코로나확산을 막다 ” 현황현재까지 코로나의 방역대책으로K방역이라는 말이 나올 정도로 대처를 잘 해오고 있다 정부의료진자영업자•특정 업종전 국민의 노력과 희생이 있었다 하지만년 이상의 지속가능한 코로나방역대책이 필요하고코로나종식의 불확실성과 미래의 새로운 감염병 대책을 준비하여야 할 것이다 현재의 코로나방역 상황제대로 단속하지 않고부과되지 않는 유명무실한 마스크쓰기방역수칙위반만원의 과태료와 국민의 자발적 참여인 권고로년 이상의 전 국민 자유와 일상생활을 규제한다 그리고 반복되는 폭풍전야

In [301]:
doc_nouns_list = [komoran_tokenizer(doc) for doc in data['summary_content_end']]

In [302]:
tfidf_vectorizer = TfidfVectorizer(min_df=3,stop_words=['지금',"요약","인지","다기","부분","드리","일부","기간","가져오","대통령","청원","해주","통하","후에","지내","줄이","이나","위하","등등","시키","나오",'따르','인해','때문','문제','당장','청원','청와대'])  # 한 번만 나타나는 단어들은 무시
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

words = np.array(tfidf_vectorizer.get_feature_names())  # 단어 추출 

In [303]:
id2vocab = {tfidf_vectorizer.vocabulary_[token]:token for token in tfidf_vectorizer.vocabulary_.keys()}
curr_doc, result = tfidf_matrix[0],[]

In [304]:
for idx, el in zip(curr_doc.indices, curr_doc.data):
    result.append((id2vocab[idx],el))

In [305]:
sorted(result, key = lambda x:x[0], reverse = True)

[('희생', 0.011443088902331559),
 ('흡연', 0.006824820006623369),
 ('효과', 0.09807355320514574),
 ('활용', 0.023527155987439884),
 ('활성', 0.0065212318302305325),
 ('활동', 0.020106252170953354),
 ('확진', 0.1506114221526126),
 ('확산', 0.031365911144472874),
 ('홍보', 0.0213206048765247),
 ('현행', 0.006824820006623369),
 ('현재', 0.08556443224839172),
 ('허가', 0.005448530733619208),
 ('행사', 0.004568790491745742),
 ('해소', 0.02608492732092213),
 ('해당', 0.003835482431638981),
 ('해결', 0.004505499723855808),
 ('하락', 0.006273182228894574),
 ('필요', 0.027328292258605644),
 ('피해', 0.0662382805999995),
 ('평균', 0.012126918558475875),
 ('특정', 0.05743377884401284),
 ('투명', 0.0065212318302305325),
 ('투기', 0.004856277815595488),
 ('코로나', 0.08922548020792594),
 ('카페', 0.005221252622182986),
 ('친절', 0.006824820006623369),
 ('취지', 0.0057215444511657795),
 ('출입', 0.006273182228894574),
 ('추후', 0.006273182228894574),
 ('추천', 0.013649640013246738),
 ('추가', 0.009010999447711616),
 ('최소', 0.011443088902331559),
 ('최고', 0.00627

In [306]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 100)
vecs = svd.fit_transform(tfidf_matrix)

In [307]:
topn = 5

dic = {}

for i in range(20):
    query_doc_vec = vecs[i]
    query_vec_norm = np.linalg.norm(query_doc_vec)
    query_unit_vec = query_doc_vec / query_vec_norm
    
    query_sentence = data.title[i]
    scores = np.dot(vecs, query_unit_vec)
    dic[query_sentence] = sorted(zip(data.title.values.tolist(), scores), key=lambda x: x[1], reverse=True)[1:topn + 1]

In [308]:
dic

{'코로나19 거리두기 체계 개선해주세요 (코로나19 전쟁의 승리)': [('대중음악 공연(콘서트) 방역지침을 변경해 주세요.',
   0.5799466960339247),
  ('엄벌해주세요', 0.5761516625519538),
  ('정부의 코로나19방역 수칙과 반대로 직원들을 한군데 집합시켜 업무하게 하는 회사 정책을 늦춰주세요.',
   0.47091048315498596),
  ('자영업을 두번 죽이는 새로운 거리두기 개편안을 반대합니다!!', 0.46406661875360233),
  ('온누리상품권의 제도적헛점', 0.1961131431270498)],
 '철도공사하면 멀쩡하던 주택도 노후주택이 된다네요!!!': [('과천 3기신도시 주택 사기분양 업자를 고발합니다',
   0.45471831111815186),
  ('부동산(주택)문제 해결방법   제안자:  *** (2021.3.16.)', 0.42167073892935103),
  ('**초교 앞 지하철 공사 착굴(환기구)위치 변경 요청', 0.3297765985906121),
  ('박상혁의원 외 15인 국회의원이 발의한 빈집 및 소규모 주택 정비에 관한 특례법 일부개정법률안 (의안번호 8637) 즉시 철회바랍니다.',
   0.2875163308652058),
  ('LH는 신혼희망타운 분양세대 건축비로 행복주택 건축하는 부분을 정확히 설명해라!!', 0.2581635333353176)],
 '기계설비유지관리자  5년 유예기간 (기존사업소 ) 시행력 철거 요구': [('기계설비유지관리자 관련 기계설비법 시행규칙 부칙 제2조 제2항의 철회를 요청합니다.',
   0.9173011444886952),
  ('탈원전 정책은 국민을 속이고 국가를 망치는 범죄행위입니다.', 0.14108822987334552),
  ('경찰간부 갑질에 매일매일 분신자살하고 싶다는 생각을 합니다.  너무억울합니다. 살려주세요', 0.11168841519506308),
  ('농아인 성폭력범죄자의 ****센터장 임

In [325]:
from sklearn.decomposition import NMF

vector_array = tfidf_matrix.toarray()
nmf = NMF(n_components = 38, random_state = 20171490)  # 군집화
nmf.fit(vector_array)

features = nmf.transform(vector_array)

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

df_features = pd.DataFrame(norm_features,index=data['title'].tolist())

In [326]:
components_df = pd.DataFrame(nmf.components_, columns=words)

In [327]:
topic_lst = []
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    topic_df = pd.DataFrame(tmp.nlargest(10))
    topic_df.columns = ['value']
    topic_lst.append(topic_df)

In [328]:
topic_lst

[       value
 학생  7.125916
 학교  5.184119
 교수  2.552357
 체결  1.078019
 폭력  1.073561
 사실  0.957057
 방관  0.913645
 의견  0.897773
 통합  0.831710
 학과  0.814693,
         value
 역사   0.782970
 드라마  0.770897
 왜곡   0.370647
 작가   0.267219
 방영   0.263334
 중국   0.242945
 마사   0.229116
 인물   0.190079
 장면   0.146419
 모독   0.124238,
          value
 교사    1.290058
 보육    0.784695
 검사    0.458419
 어린이집  0.398036
 아이들   0.336953
 시간    0.288081
 코로나   0.253192
 등원    0.141128
 근무    0.120663
 학부모   0.117209,
          value
 매출    0.911150
 영업    0.626717
 지원금   0.535037
 제한    0.475602
 년대    0.355163
 소상공인  0.329936
 비교    0.307238
 버팀목   0.258751
 재난    0.254431
 플러스   0.205421,
         value
 분양   1.626277
 주택   1.259921
 임대   1.142929
 아파트  0.540281
 세대   0.395276
 조기   0.358320
 사업자  0.354007
 입주   0.312329
 임차인  0.271789
 상환   0.263687,
         value
 백신   1.506867
 접종   1.283331
 격리   0.328354
 가짜   0.278327
 면제   0.217001
 문재인  0.216809
 음성   0.201312
 코로나  0.192332
 논란   0.183541
 방문   0.1

In [329]:
df_features = pd.DataFrame(norm_features,index=data['title'].tolist())

def similarity_print(article_title):
    article_category = data[data['title']==article_title]['category'].values
    print("# ", article_title, article_category )
    article = df_features.loc[article_title]

    similarities = df_features.dot(article)
    top = similarities.nlargest(3)

    count = 1

    for i in range(1, len(top)):
        
        if top[i] > 0.9:
            similar_article_category = data[data['title']==top.index[i]]['category'].values

            print(f"{count}: "+top.index[i]+'\n Similarities'+str(top[i]))

            count+=1 

    if count==1:    

        print('유사한 청원이 없습니다.')

In [330]:
title_lst = data[data['progress']=='청원진행중 '].title.values.tolist()

In [331]:
for title in title_lst:
    article_title = title
    similarity_print(article_title)

#  코로나19 거리두기 체계 개선해주세요 (코로나19 전쟁의 승리) ['보건복지']
1: 엄벌해주세요
 Similarities0.9892370088063093
2: 대중음악 공연(콘서트) 방역지침을 변경해 주세요.
 Similarities0.9864045924660656
#  철도공사하면 멀쩡하던 주택도 노후주택이 된다네요!!! ['교통/건축/국토']
1: 지중하 공사 저지 건
 Similarities0.9452039188018099
2: "국지도 82호선과 정남 산업단지로 이어지는 이상한도로"특정인과 공무원의 결탁으로 땅투기의혹
 Similarities0.9403248383946227
#  기계설비유지관리자  5년 유예기간 (기존사업소 ) 시행력 철거 요구 ['일자리']
1: 기계설비유지관리자 관련 기계설비법 시행규칙 부칙 제2조 제2항의 철회를 요청합니다.
 Similarities0.9997556841480519
#  금감원의 주식 자전거래 감독 강화를 요청드립니다 ['경제민주화']
유사한 청원이 없습니다.
#  LH와 관련된 탐관오리들을 처벌해 주시고 재산은 국고에 환수하길 원합니다. ['기타']
1: 독도는 우리땅을 독도는 대한민국 땅 으로 표현을 바꿔주세요
 Similarities0.9967667850461399
2: 현역 대한민국 사관생도가 우국충정으로 대통령님께 고언을 올립니다.
 Similarities0.9942804630253843
#  '정치부기자의 폭행으로 인한 실명사건'가해자의 부인으로서 사건의 진실에 대해 알려드리고자 합니다. ['인권/성평등']
1: 노원 일가족 3명 살인사건의 가해자 20대 남성 신상공개 촉구 바랍니다.
 Similarities0.9781251389494333
2: 보이스피싱범죄의 전달책에게 강한 처벌을 간곡히 호소드려요
 Similarities0.9743134418580691
#  조상의 묘를 무분별하게 파헤치고 훼손하는 육촌 이웃을 저지하는 것을 도와주십시오! ['기타']
유사한 청원이 없습니다.
#  누구를위한

1: 공공간호사 제도를 반대합니다.
 Similarities0.9408096392075466
2: 의료진에 무책임함으로 젊은 제조카에 억울함을 밝혀주세요
 Similarities0.9393879734960691
#  **글로벌 청원합니다. ['기타']
유사한 청원이 없습니다.
#  경매를 취하하게 해주세요!! (무너진 금융시스템 재조사 및 처벌 촉구) ['기타']
유사한 청원이 없습니다.
#  한 순간에 모두 불타 버렸습니다. ['농산어촌']
1: **화재 노조탄압 제발 막아주세요
 Similarities0.9962069088486509
2: **화재 평사원협의회의 노조인가서를 반려해주시기 바랍니다.
 Similarities0.9906821550267189
#  *** 관련 법규가 필요합니다. ['안전/환경']
1: 미곡 부산물 쌀겨(미강), 왕겨틀 페기물관리법에서 제외(개정)요청
 Similarities0.9854977261977296
2: 오토바이배달대행(특수고용직)에 미성년자 고용(계약)금지
 Similarities0.9433966244422167
#  종합건설업체에서 협력업체에게 행하는 갑질/횡포가 근절되어 선진문화가 자리 잡을 수 있도록 도와주세요. ['인권/성평등']
유사한 청원이 없습니다.
#  어린이 운송용 차량의 썬팅 단속을 하지말아 주세요!! ['교통/건축/국토']
1: 어린이집 차량 썬팅을 벗기지 않길 바랍니다.
 Similarities0.9801347918216043
2: 어린이보호차량 관련 차량검사 강화에 대하여,,,,
 Similarities0.9648086758949934
#  코로나 시대로 어려운 상장기업들 내년까지 상장폐지 유보해주시길 바랍니다. ['미래']
1: 제도개선을 통해 블록체인의 미래를 열어주세요
 Similarities0.9363456672809906
2: 탈원전 정책은 국민을 속이고 국가를 망치는 범죄행위입니다.
 Similarities0.9257964628529357
#  경미한 교통사고 입원(한의원) 못하게 해 

#  장애인 활동지원 서비스 가족이 가능하도록 개정 부탁드립니다. ['인권/성평등']
1: 장애인 활동보조사 직계가족 확대하고 발달장애에 대한 기준 마련해주세요.
 Similarities0.9976914055405691
2: 국민여러분! 거리로 내몰린 74명의 중증장애인들을  도와주십시오.
 Similarities0.9929199919030435
#  국민여러분! 거리로 내몰린 74명의 중증장애인들을  도와주십시오. ['보건복지']
1: 장애인 활동보조사 직계가족 확대하고 발달장애에 대한 기준 마련해주세요.
 Similarities0.997541785981454
2: 장애인 활동지원 서비스 가족이 가능하도록 개정 부탁드립니다.
 Similarities0.9929199919030435
#  장애인 활동보조사 직계가족 확대하고 발달장애에 대한 기준 마련해주세요. ['보건복지']
1: 장애인 활동지원 서비스 가족이 가능하도록 개정 부탁드립니다.
 Similarities0.9976914055405691
2: 국민여러분! 거리로 내몰린 74명의 중증장애인들을  도와주십시오.
 Similarities0.997541785981454
#  환자를 안일하게 진찰한 산부인과 당직의로 인해 출산 후 자간증을 앓았습니다 ['기타']
1: 간호사 들들 볶는 의료기관 인증 평가 제도 변화를 요구합니다.  암기 잘하는 간호사가 있는 곳은 1등급 패스~!
 Similarities0.9914899356908029
2: 의료진에 무책임함으로 젊은 제조카에 억울함을 밝혀주세요
 Similarities0.9894028183616036
#  4차 재난지원금 버팀목자금 억울합니다. 2019년도 오픈한 개인사업자입니다. ['행정']
1: 4차재난지원금 소상공인 버팀목자금 플러스
 Similarities0.9978466692811621
2: 4차 버팀목자금에 관하여...
 Similarities0.9964804644338883
#  소상공인4차재난지원금지급기준 억울합니다 ['행정']
1: 4차 버팀목자

LSA 의 성능이 더 좋음

### nmf cross validation

In [341]:
'''
from surprise.model_selection import cross_validate

algo = NMF(n_components=35, solver="mu")
cv_results = cross_validate(algo, tfidf_matrix, measures=['RMSE'], cv=5, verbose=False)
'''

'\nfrom surprise.model_selection import cross_validate\n\nalgo = NMF(n_components=35, solver="mu")\ncv_results = cross_validate(algo, tfidf_matrix, measures=[\'RMSE\'], cv=5, verbose=False)\n'

In [316]:
'''
# Create a DataFrame: components_df
components_df = pd.DataFrame(nmf.components_, columns=words)

topic_lst = []
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    topic_df = pd.DataFrame(tmp.nlargest(10))
    topic_df.columns = ['value']
    topic_lst.append(topic_df)
    
topic_lst
'''